In [ ]:
# Install memorizz and required dependencies
%pip install -qU memorizz

# Install Oracle database driver (required for Oracle provider)
%pip install -qU oracledb

# Install OpenAI SDK (for LLM and embeddings)
%pip install -qU openai

# Install requests (for tool examples like weather API)
%pip install -qU requests

# Install python-dotenv for .env file support (optional but recommended)
%pip install -qU python-dotenv

print("✅ All packages installed successfully!")


In [ ]:
import os

os.environ["ORACLE_USER"] = ""
os.environ["ORACLE_PASSWORD"] = "VVxLC!"
os.environ["ORACLE_DSN"] = ""


In [ ]:
import os

# Get credentials from environment variables with defaults
ORACLE_USER = os.getenv("ORACLE_USER", "test")
ORACLE_PASSWORD = os.getenv("ORACLE_PASSWORD", "test")
ORACLE_DSN = os.getenv("ORACLE_DSN", "test")

Place the need to run oracle in thick mode and the step that takes to make that happen and also place in the benfits.


In [ ]:
import oracledb

print("Connecting to Oracle...")
print(f"  User: {ORACLE_USER}")
print(f"  DSN: {ORACLE_DSN}")

# Enabling python-oracledb Thick mode. 
# Might need to add instructions for the developers that point to INSTALL_ORACLE_CLIENT.md
# Use the x86_64 version (matches your Python environment)
oracledb.init_oracle_client(
    lib_dir=os.path.expanduser("~/oracle/instantclient_19_16_x86_64")
)

conn = oracledb.connect(user=ORACLE_USER, password=ORACLE_PASSWORD, dsn=ORACLE_DSN)
cursor = conn.cursor()
print("✓ Connected successfully!")


In [ ]:
  from memorizz.cli import setup_oracle
  setup_oracle()

In [ ]:
import logging
import os

# Configure logging for Jupyter notebook
os.environ['MEMORIZZ_LOG_LEVEL'] = 'INFO'

# Set up proper logging configuration for notebooks
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True  # This overwrites any existing configuration
)

In [ ]:
import getpass

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API key: ")

In [ ]:
from memorizz.memory_provider.oracle import OracleProvider, OracleConfig

# Create Oracle configuration
oracle_config = OracleConfig(
    user=ORACLE_USER, 
    password=ORACLE_PASSWORD,
    dsn=ORACLE_DSN,
    lazy_vector_indexes=False,
    embedding_provider="openai",
    embedding_config={
        "model": "text-embedding-3-small",
        "api_key": os.getenv("OPENAI_API_KEY"),
    }
)

# Create Oracle Memory provider
oracle_memory_provider = OracleProvider(oracle_config)
print("✓ Oracle provider initialized!")



In [ ]:
from memorizz.memagent.builders import MemAgentBuilder

agent_builder_made = (MemAgentBuilder()
    # 1. Core identity
    .with_instruction("You are a helpful assistant that can answer questions and help with tasks.")
    # 2. Infrastructure
    .with_memory_provider(oracle_memory_provider)
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY"),
    })
    .build()
)


In [ ]:
agent_builder_made.save()

In [ ]:
response = agent_builder_made.run("Hello! My name is Alice and I love hiking in the mountains.")
print(f"Agent: {response}\n")


In [ ]:
response2 = agent_builder_made.run("What was my name again?")
print(f"Agent: {response2}\n")
